In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
SCRIPT_ALL_SCHEMA = "all_schema"
SCRIPT_SINGLE_SCHEMA = "single_schema"
SCRIPT_NO_SCHEMA = "no_schema"

In [ ]:
import yaml

with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

MANAGED_SCHEMA = False
ENV = config["app_settings"]['environment']
DATABASE_NAME = config["app_settings"]['database_name']

script_type = config["script_mode"]["type"]

print(f"{script_type} | {ENV} | {DATABASE_NAME}")

if script_type == SCRIPT_ALL_SCHEMA:
    EXCLUDE_SCHEMA = config["script_mode"]["all_schema"]["schemas_to_exclude"]
    MANAGED_SCHEMA = config["script_mode"]["all_schema"]["create_managed_schema"]
    print(f"{EXCLUDE_SCHEMA} | {MANAGED_SCHEMA}")
elif script_type == SCRIPT_SINGLE_SCHEMA:
    TARGET_SCHEMA_NAME = config["script_mode"]["single_schema"]["schema_name"]
    print(f"{TARGET_SCHEMA_NAME} | {MANAGED_SCHEMA}")
elif script_type == SCRIPT_NO_SCHEMA:
    print("Running without schema-level context")


EXECUTE_SCRIPT = True

In [ ]:
DB_ROLE_TYPE_READ = "RO"
DB_ROLE_TYPE_WRITE = "RW"
DB_ROLE_TYPE_FULL = "ALL"
DB_ROLE_TYPE_OWNER = "OWNER"
DB_ROLE_TYPE_SHARE = "SHARE"

DB_WAGERING_WHS = f"{ENV}_WAGERING_WHS_DB"

NEW_LINE = "\n"

In [ ]:
SNOWFLAKE_SELECT_ITEMS = ["TABLES", "VIEWS", "EXTERNAL TABLES", "DYNAMIC TABLES", "ICEBERG TABLES", "MATERIALIZED VIEWS", "STREAMS"]
SNOWFLAKE_SHARE_ITEMS = ["TABLES", "EXTERNAL TABLES", "DYNAMIC TABLES", "ICEBERG TABLES"]
SNOWFLAKE_REFERENCES_ITEMS = ["TABLES", "VIEWS", "EXTERNAL TABLES", "ICEBERG TABLES", "MATERIALIZED VIEWS"]

SNOWFLAKE_USAGE_ITEMS = ["FUNCTIONS", "PROCEDURES", "FILE FORMATS", "STAGES", "SEQUENCES"]
SNOWFLAKE_MONITOR_ITEMS = ["TASKS", "ALERTS", "DYNAMIC TABLES"]
SNOWFLAKE_READ_ITEMS = ["STAGES"]

SNOWFLAKE_ETL_ITEMS = ["TABLES", "ICEBERG TABLES"]
SNOWFLAKE_WRITE_ITEMS = ["STAGES"]
SNOWFLAKE_OPERATE_ITEMS = ["DYNAMIC TABLES", "ALERTS", "TASKS"]
SNOWFLAKE_ETL_MONITOR_ITEMS = ["PIPES"]


In [ ]:
USE ROLE USERADMIN;

In [ ]:
ROLE_DATAPRODUCT_OWNER = f'{ENV}_ORG_DPOWNER'

sql_script = f"CREATE ROLE IF NOT EXISTS {ROLE_DATAPRODUCT_OWNER};" 

print(sql_script)
if EXECUTE_SCRIPT:
    session.sql("ALTER SESSION SET MULTI_STATEMENT_COUNT = 0").collect()
    session.sql(sql_script).collect()


In [ ]:
ROLE_DATABASE_ADMIN = f'{ENV}_DATABASE_ADMIN'

sql_script = f"CREATE ROLE IF NOT EXISTS {ROLE_DATABASE_ADMIN};" 

print(sql_script)
if EXECUTE_SCRIPT:
    session.sql("ALTER SESSION SET MULTI_STATEMENT_COUNT = 0").collect()
    session.sql(sql_script).collect()
    

In [ ]:
ACCESS_ROLE_DB_READ = f"{ENV}_AR_ALLDB_{DB_ROLE_TYPE_READ}"
ACCESS_ROLE_DB_WRITE = f"{ENV}_AR_ALLDB_{DB_ROLE_TYPE_WRITE}"
ACCESS_ROLE_DB_FULL = f"{ENV}_AR_ALLDB_{DB_ROLE_TYPE_FULL}"
ACCESS_ROLE_DBT_ETL = f"{ENV}_AR_DBT_ETL"

sql_script = ""

sql_script += f"CREATE ROLE IF NOT EXISTS {ACCESS_ROLE_DB_READ};{NEW_LINE}" 
sql_script += f"CREATE ROLE IF NOT EXISTS {ACCESS_ROLE_DB_WRITE};{NEW_LINE}" 
sql_script += f"CREATE ROLE IF NOT EXISTS {ACCESS_ROLE_DB_FULL};{NEW_LINE}" 

print(sql_script)
if EXECUTE_SCRIPT:
    print("Executing SQL Script")
    session.sql("ALTER SESSION SET MULTI_STATEMENT_COUNT = 0").collect()
    session.sql(sql_script).collect()


In [ ]:
if script_type != SCRIPT_SINGLE_SCHEMA:
    
    sql_script = f"USE ROLE ACCOUNTADMIN;{NEW_LINE}"
    sql_script += f"GRANT OWNERSHIP ON DATABASE {DATABASE_NAME} TO ROLE {ROLE_DATABASE_ADMIN} REVOKE CURRENT GRANTS;"
    
    print(sql_script)
    if EXECUTE_SCRIPT:
        session.sql("ALTER SESSION SET MULTI_STATEMENT_COUNT = 0").collect()
        session.sql(sql_script).collect()
    

In [ ]:
session.sql(f"USE ROLE {ROLE_DATABASE_ADMIN}").collect()

In [ ]:
sql_script = f"USE DATABASE {DATABASE_NAME};{NEW_LINE}"
session.sql(sql_script).collect()

In [ ]:
if script_type != SCRIPT_NO_SCHEMA:
    rslt = session.sql(f"SHOW SCHEMAS IN DATABASE {DATABASE_NAME}").collect()
    if script_type == SCRIPT_SINGLE_SCHEMA:
        filtered_schemas = [row for row in rslt if row.name in (TARGET_SCHEMA_NAME)]
    else:
        filtered_schemas = [row for row in rslt if row.name not in (EXCLUDE_SCHEMA)]
    filtered_schemas
else:
    filtered_schemas = []

In [ ]:
DB_ROLE_NAME_READ = f"DB_{DATABASE_NAME}_{DB_ROLE_TYPE_READ}"
DB_ROLE_NAME_WRITE = f"DB_{DATABASE_NAME}_{DB_ROLE_TYPE_WRITE}"
DB_ROLE_NAME_FULL = f"DB_{DATABASE_NAME}_{DB_ROLE_TYPE_FULL}"

if script_type != SCRIPT_SINGLE_SCHEMA:

    sql_script = f"USE DATABASE {DATABASE_NAME};{NEW_LINE}"
    
    sql_script += f"CREATE DATABASE ROLE IF NOT EXISTS {DB_ROLE_NAME_READ};{NEW_LINE}" 
    sql_script += f"CREATE DATABASE ROLE IF NOT EXISTS {DB_ROLE_NAME_WRITE};{NEW_LINE}" 
    
    # Create the Database Role with all permissions on database. 
    # In the next Step Schema Role with Full permission will be assigned to this role as well
    sql_script += f"CREATE DATABASE ROLE IF NOT EXISTS {DB_ROLE_NAME_FULL};{NEW_LINE}"
    sql_script += f"GRANT ALL PRIVILEGES ON DATABASE {DATABASE_NAME} TO DATABASE ROLE {DB_ROLE_NAME_FULL} ;{NEW_LINE}" 
    
    sql_script += f"GRANT DATABASE ROLE {DB_ROLE_NAME_READ} TO ROLE {ACCESS_ROLE_DB_READ};{NEW_LINE}"
    sql_script += f"GRANT DATABASE ROLE {DB_ROLE_NAME_WRITE} TO ROLE {ACCESS_ROLE_DB_WRITE};{NEW_LINE}"
    sql_script += f"GRANT DATABASE ROLE {DB_ROLE_NAME_FULL} TO ROLE {ACCESS_ROLE_DB_FULL};{NEW_LINE}"
    
    print(sql_script)
    if EXECUTE_SCRIPT:
        print("Executing SQL Script")
        session.sql("ALTER SESSION SET MULTI_STATEMENT_COUNT = 0").collect()
        session.sql(sql_script).collect()
        

In [ ]:
if script_type != SCRIPT_NO_SCHEMA:

    for row in filtered_schemas:
        schema_name = f"{row.name}"
        schema_role_name_owner = f"SC_{schema_name}_{DB_ROLE_TYPE_OWNER}"
        schema_role_name_read = f"SC_{schema_name}_{DB_ROLE_TYPE_READ}"
        
        sql_script = f"USE ROLE {ROLE_DATABASE_ADMIN};{NEW_LINE}"
        sql_script += f"USE DATABASE {DATABASE_NAME};{NEW_LINE}"
        sql_script += f"CREATE DATABASE ROLE IF NOT EXISTS {schema_role_name_owner};{NEW_LINE}" 
    
        sql_script += f"GRANT OWNERSHIP ON SCHEMA {schema_name} TO DATABASE ROLE {schema_role_name_owner} REVOKE CURRENT GRANTS;{NEW_LINE}" 
        sql_script += f"GRANT DATABASE ROLE {schema_role_name_owner} TO ROLE {ROLE_DATAPRODUCT_OWNER};{NEW_LINE}" 
    
        sql_script += f"{NEW_LINE}{NEW_LINE}"
        print (sql_script);
    
        if EXECUTE_SCRIPT:
            print (f"--Execute script for schema {DATABASE_NAME}.{schema_name}");
            session.sql("ALTER SESSION SET MULTI_STATEMENT_COUNT = 0").collect()
            session.sql(sql_script).collect()
    

In [ ]:
#Assign Schema Owners on WAGERING_WHS_DB to DBT_ETL_ROLE
if DATABASE_NAME == DB_WAGERING_WHS:
    sql_script = f"GRANT DATABASE ROLE {DB_ROLE_NAME_FULL} TO ROLE {ACCESS_ROLE_DBT_ETL};{NEW_LINE}"
    for row in filtered_schemas:
        schema_name = f"{row.name}"
        schema_role_name_owner = f"SC_{schema_name}_{DB_ROLE_TYPE_OWNER}"
        
        sql_script += f"GRANT DATABASE ROLE {schema_role_name_owner} TO ROLE {ACCESS_ROLE_DBT_ETL};{NEW_LINE}" 
    
    if EXECUTE_SCRIPT:
        print (f"--Execute script to assign Schema Owner roles on {DB_WAGERING_WHS} to {ACCESS_ROLE_DBT_ETL}");
        print (sql_script);
        session.sql("ALTER SESSION SET MULTI_STATEMENT_COUNT = 0").collect()
        session.sql(sql_script).collect()
    

In [ ]:

if script_type == SCRIPT_NO_SCHEMA:
    sql_script = f"USE ROLE {ROLE_DATABASE_ADMIN};{NEW_LINE}"
    sql_script += f"USE DATABASE {DATABASE_NAME};{NEW_LINE}"

    sql_script += f"GRANT USAGE ON DATABASE {DATABASE_NAME} TO DATABASE ROLE {DB_ROLE_NAME_READ};{NEW_LINE}" 
    sql_script += f"GRANT USAGE ON ALL SCHEMAS IN DATABASE {DATABASE_NAME} TO DATABASE ROLE {DB_ROLE_NAME_READ};{NEW_LINE}" 
    
    for type in SNOWFLAKE_SELECT_ITEMS:
        sql_script += f"GRANT SELECT ON ALL {type} IN DATABASE {DATABASE_NAME} TO DATABASE ROLE {DB_ROLE_NAME_READ};{NEW_LINE}"
        sql_script += f"GRANT SELECT ON FUTURE {type} IN DATABASE {DATABASE_NAME} TO DATABASE ROLE {DB_ROLE_NAME_READ};{NEW_LINE}"

    for type in SNOWFLAKE_REFERENCES_ITEMS:
        sql_script += f"GRANT REFERENCES ON ALL {type} IN DATABASE {DATABASE_NAME} TO DATABASE ROLE {DB_ROLE_NAME_READ};{NEW_LINE}"
        sql_script += f"GRANT REFERENCES ON FUTURE {type} IN DATABASE {DATABASE_NAME} TO DATABASE ROLE {DB_ROLE_NAME_READ};{NEW_LINE}"

    for type in SNOWFLAKE_USAGE_ITEMS:
        sql_script += f"GRANT USAGE ON ALL {type} IN DATABASE {DATABASE_NAME} TO DATABASE ROLE {DB_ROLE_NAME_READ};{NEW_LINE}"
        sql_script += f"GRANT USAGE ON FUTURE {type} IN DATABASE {DATABASE_NAME} TO DATABASE ROLE {DB_ROLE_NAME_READ};{NEW_LINE}"

    for type in SNOWFLAKE_MONITOR_ITEMS:
        sql_script += f"GRANT MONITOR ON ALL {type} IN DATABASE {DATABASE_NAME} TO DATABASE ROLE {DB_ROLE_NAME_READ};{NEW_LINE}"
        sql_script += f"GRANT MONITOR ON FUTURE {type} IN DATABASE {DATABASE_NAME} TO DATABASE ROLE {DB_ROLE_NAME_READ};{NEW_LINE}"

    for type in SNOWFLAKE_READ_ITEMS:
        sql_script += f"GRANT READ ON ALL {type} IN DATABASE {DATABASE_NAME} TO DATABASE ROLE {DB_ROLE_NAME_READ};{NEW_LINE}"
        sql_script += f"GRANT READ ON FUTURE {type} IN DATABASE {DATABASE_NAME} TO DATABASE ROLE {DB_ROLE_NAME_READ};{NEW_LINE}"

    sql_script += f"{NEW_LINE}{NEW_LINE}"
    print (f"--Starting generating script for readonly");
    print (sql_script);

    
    if EXECUTE_SCRIPT:
        
        print (f"--Execute script for readonly");
        session.sql("ALTER SESSION SET MULTI_STATEMENT_COUNT = 0").collect()
        session.sql(sql_script).collect()
else:
    for row in filtered_schemas:
        schema_name = row.name
        schema_role_name_read = f"SC_{schema_name}_{DB_ROLE_TYPE_READ}"
    
        sql_script = f"USE ROLE {ROLE_DATABASE_ADMIN};{NEW_LINE}"
        sql_script += f"USE DATABASE {DATABASE_NAME};{NEW_LINE}"
        sql_script += f"CREATE DATABASE ROLE IF NOT EXISTS {schema_role_name_read};{NEW_LINE}" 
        sql_script += f"GRANT DATABASE ROLE {schema_role_name_read} TO DATABASE ROLE {DB_ROLE_NAME_READ};{NEW_LINE}" 
    
        sql_script += f"GRANT USAGE ON DATABASE {DATABASE_NAME} TO DATABASE ROLE {schema_role_name_read};{NEW_LINE}" 
        sql_script += f"GRANT USAGE ON SCHEMA {schema_name} TO DATABASE ROLE {schema_role_name_read};{NEW_LINE}" 
        
        for type in SNOWFLAKE_SELECT_ITEMS:
            sql_script += f"GRANT SELECT ON ALL {type} IN SCHEMA {DATABASE_NAME}.{schema_name} TO DATABASE ROLE {schema_role_name_read};{NEW_LINE}"
            sql_script += f"GRANT SELECT ON FUTURE {type} IN SCHEMA {DATABASE_NAME}.{schema_name} TO DATABASE ROLE {schema_role_name_read};{NEW_LINE}"
    
        for type in SNOWFLAKE_REFERENCES_ITEMS:
            sql_script += f"GRANT REFERENCES ON ALL {type} IN SCHEMA {DATABASE_NAME}.{schema_name} TO DATABASE ROLE {schema_role_name_read};{NEW_LINE}"
            sql_script += f"GRANT REFERENCES ON FUTURE {type} IN SCHEMA {DATABASE_NAME}.{schema_name} TO DATABASE ROLE {schema_role_name_read};{NEW_LINE}"
    
        for type in SNOWFLAKE_USAGE_ITEMS:
            sql_script += f"GRANT USAGE ON ALL {type} IN SCHEMA {DATABASE_NAME}.{schema_name} TO DATABASE ROLE {schema_role_name_read};{NEW_LINE}"
            sql_script += f"GRANT USAGE ON FUTURE {type} IN SCHEMA {DATABASE_NAME}.{schema_name} TO DATABASE ROLE {schema_role_name_read};{NEW_LINE}"
    
        for type in SNOWFLAKE_MONITOR_ITEMS:
            sql_script += f"GRANT MONITOR ON ALL {type} IN SCHEMA {DATABASE_NAME}.{schema_name} TO DATABASE ROLE {schema_role_name_read};{NEW_LINE}"
            sql_script += f"GRANT MONITOR ON FUTURE {type} IN SCHEMA {DATABASE_NAME}.{schema_name} TO DATABASE ROLE {schema_role_name_read};{NEW_LINE}"
    
        for type in SNOWFLAKE_READ_ITEMS:
            sql_script += f"GRANT READ ON ALL {type} IN SCHEMA {DATABASE_NAME}.{schema_name} TO DATABASE ROLE {schema_role_name_read};{NEW_LINE}"
            sql_script += f"GRANT READ ON FUTURE {type} IN SCHEMA {DATABASE_NAME}.{schema_name} TO DATABASE ROLE {schema_role_name_read};{NEW_LINE}"
    
        sql_script += f"{NEW_LINE}{NEW_LINE}"
        print (f"--Starting generating script for schema {DATABASE_NAME}.{schema_name}");
        print (sql_script);
    
        
        if EXECUTE_SCRIPT:
            
            print (f"--Execute script for schema {DATABASE_NAME}.{schema_name}");
            session.sql("ALTER SESSION SET MULTI_STATEMENT_COUNT = 0").collect()
            session.sql(sql_script).collect()
    

In [ ]:
if script_type != SCRIPT_NO_SCHEMA:

    for row in filtered_schemas:
        schema_name = row.name
        schema_role_name_share = f"SC_{schema_name}_{DB_ROLE_TYPE_SHARE}"
    
        sql_script = f"USE ROLE {ROLE_DATABASE_ADMIN};{NEW_LINE}"
        sql_script += f"USE DATABASE {DATABASE_NAME};{NEW_LINE}"
        sql_script += f"CREATE DATABASE ROLE IF NOT EXISTS {schema_role_name_share};{NEW_LINE}" 
    
        sql_script += f"GRANT USAGE ON DATABASE {DATABASE_NAME} TO DATABASE ROLE {schema_role_name_share};{NEW_LINE}" 
        sql_script += f"GRANT USAGE ON SCHEMA {schema_name} TO DATABASE ROLE {schema_role_name_share};{NEW_LINE}" 
        
        for type in SNOWFLAKE_SHARE_ITEMS:
            sql_script += f"GRANT SELECT ON ALL {type} IN SCHEMA {DATABASE_NAME}.{schema_name} TO DATABASE ROLE {schema_role_name_share};{NEW_LINE}"
    
        sql_view_list = f"""SELECT TABLE_NAME 
            FROM {DATABASE_NAME}.INFORMATION_SCHEMA.VIEWS
            WHERE TABLE_SCHEMA = '{schema_name}' 
            AND TABLE_SCHEMA != 'INFORMATION_SCHEMA'
            AND IS_SECURE
        """
        rslt = session.sql(sql_view_list).collect()
        for secure_view in rslt:
            sql_script += f"GRANT SELECT ON VIEW {DATABASE_NAME}.{schema_name}.{secure_view.TABLE_NAME} TO DATABASE ROLE {DATABASE_NAME}.{schema_role_name_share};"
    
        sql_script += f"{NEW_LINE}{NEW_LINE}"
        print (f"--Starting generating script for schema {DATABASE_NAME}.{schema_name}");
        print (sql_script);
    
        
        if EXECUTE_SCRIPT:
            
            print (f"--Execute script for schema {DATABASE_NAME}.{schema_name}");
            session.sql("ALTER SESSION SET MULTI_STATEMENT_COUNT = 0").collect()
            session.sql(sql_script).collect()
    

In [ ]:
if script_type == SCRIPT_NO_SCHEMA:

    sql_script = f"USE ROLE {ROLE_DATABASE_ADMIN};{NEW_LINE}"
    sql_script += f"USE DATABASE {DATABASE_NAME};{NEW_LINE}"

    for type in SNOWFLAKE_ETL_ITEMS:
        sql_script += f"GRANT INSERT, UPDATE, DELETE, TRUNCATE ON ALL {type} IN DATABASE {DATABASE_NAME} TO DATABASE ROLE {DB_ROLE_NAME_WRITE};{NEW_LINE}"
        sql_script += f"GRANT INSERT, UPDATE, DELETE, TRUNCATE ON FUTURE {type} IN DATABASE {DATABASE_NAME} TO DATABASE ROLE {DB_ROLE_NAME_WRITE};{NEW_LINE}"

    for type in SNOWFLAKE_WRITE_ITEMS:
        sql_script += f"GRANT READ, WRITE ON ALL {type} IN DATABASE {DATABASE_NAME} TO DATABASE ROLE {DB_ROLE_NAME_WRITE};{NEW_LINE}"
        sql_script += f"GRANT READ, WRITE ON FUTURE {type} IN DATABASE {DATABASE_NAME} TO DATABASE ROLE {DB_ROLE_NAME_WRITE};{NEW_LINE}"

    for type in SNOWFLAKE_OPERATE_ITEMS:
        sql_script += f"GRANT OPERATE ON ALL {type} IN DATABASE {DATABASE_NAME} TO DATABASE ROLE {DB_ROLE_NAME_WRITE};{NEW_LINE}"
        sql_script += f"GRANT OPERATE ON FUTURE {type} IN DATABASE {DATABASE_NAME} TO DATABASE ROLE {DB_ROLE_NAME_WRITE};{NEW_LINE}"

    for type in SNOWFLAKE_ETL_MONITOR_ITEMS:
        sql_script += f"GRANT MONITOR ON FUTURE {type} IN DATABASE {DATABASE_NAME} TO DATABASE ROLE {DB_ROLE_NAME_WRITE};{NEW_LINE}"

    sql_script += f"{NEW_LINE}{NEW_LINE}"
    print (f"--Starting generating script for write");
    print (sql_script);

    if EXECUTE_SCRIPT:
        print (f"--Execute script for write");
        session.sql("ALTER SESSION SET MULTI_STATEMENT_COUNT = 0").collect()
        session.sql(sql_script).collect()
else:

    for row in filtered_schemas:
        schema_name = row.name
        schema_role_name_write = f"SC_{schema_name}_{DB_ROLE_TYPE_WRITE}"
        schema_role_name_read = f"SC_{schema_name}_{DB_ROLE_TYPE_READ}"
    
        sql_script = f"USE ROLE {ROLE_DATABASE_ADMIN};{NEW_LINE}"
        sql_script += f"USE DATABASE {DATABASE_NAME};{NEW_LINE}"
        sql_script += f"CREATE DATABASE ROLE IF NOT EXISTS {schema_role_name_write};{NEW_LINE}" 
        
        sql_script += f"GRANT DATABASE ROLE {schema_role_name_read} TO DATABASE ROLE {schema_role_name_write} ;{NEW_LINE}" 
        sql_script += f"GRANT DATABASE ROLE {schema_role_name_write} TO DATABASE ROLE {DB_ROLE_NAME_WRITE};{NEW_LINE}" 
    
        for type in SNOWFLAKE_ETL_ITEMS:
            sql_script += f"GRANT INSERT, UPDATE, DELETE, TRUNCATE ON ALL {type} IN SCHEMA {DATABASE_NAME}.{schema_name} TO DATABASE ROLE {schema_role_name_write};{NEW_LINE}"
            sql_script += f"GRANT INSERT, UPDATE, DELETE, TRUNCATE ON FUTURE {type} IN SCHEMA {DATABASE_NAME}.{schema_name} TO DATABASE ROLE {schema_role_name_write};{NEW_LINE}"
    
        for type in SNOWFLAKE_WRITE_ITEMS:
            sql_script += f"GRANT READ, WRITE ON ALL {type} IN SCHEMA {DATABASE_NAME}.{schema_name} TO DATABASE ROLE {schema_role_name_write};{NEW_LINE}"
            sql_script += f"GRANT READ, WRITE ON FUTURE {type} IN SCHEMA {DATABASE_NAME}.{schema_name} TO DATABASE ROLE {schema_role_name_write};{NEW_LINE}"
    
        for type in SNOWFLAKE_OPERATE_ITEMS:
            sql_script += f"GRANT OPERATE ON ALL {type} IN SCHEMA {DATABASE_NAME}.{schema_name} TO DATABASE ROLE {schema_role_name_write};{NEW_LINE}"
            sql_script += f"GRANT OPERATE ON FUTURE {type} IN SCHEMA {DATABASE_NAME}.{schema_name} TO DATABASE ROLE {schema_role_name_write};{NEW_LINE}"
    
        for type in SNOWFLAKE_ETL_MONITOR_ITEMS:
            sql_script += f"GRANT MONITOR ON FUTURE {type} IN SCHEMA {DATABASE_NAME}.{schema_name} TO DATABASE ROLE {schema_role_name_write};{NEW_LINE}"
    
        sql_script += f"{NEW_LINE}{NEW_LINE}"
        print (f"--Starting generating script for schema {schema_name}");
        print (sql_script);
    
        if EXECUTE_SCRIPT:
            print (f"--Execute script for schema {DATABASE_NAME}.{schema_name}");
            session.sql("ALTER SESSION SET MULTI_STATEMENT_COUNT = 0").collect()
            session.sql(sql_script).collect()
    

In [ ]:
if script_type == SCRIPT_NO_SCHEMA:

    sql_script = f"USE ROLE {ROLE_DATABASE_ADMIN};{NEW_LINE}"
    sql_script += f"USE DATABASE {DATABASE_NAME};{NEW_LINE}"
    sql_script += f"GRANT ALL PRIVILEGES ON ALL SCHEMAS IN DATABASE {DATABASE_NAME} TO DATABASE ROLE {DB_ROLE_NAME_FULL} ;{NEW_LINE}" 

    sql_script += f"{NEW_LINE}{NEW_LINE}"
    print (sql_script);

    if EXECUTE_SCRIPT:
        print (f"--Executing script for all");
        session.sql("ALTER SESSION SET MULTI_STATEMENT_COUNT = 0").collect()
        session.sql(sql_script).collect()
else:
    for row in filtered_schemas:
        schema_name = f"{row.name}"
        schema_role_name_full = f"SC_{schema_name}_{DB_ROLE_TYPE_FULL}"
        schema_role_name_write = f"SC_{schema_name}_{DB_ROLE_TYPE_WRITE}"
        
        sql_script = f"USE ROLE {ROLE_DATABASE_ADMIN};{NEW_LINE}"
        sql_script += f"USE DATABASE {DATABASE_NAME};{NEW_LINE}"
        sql_script += f"CREATE DATABASE ROLE IF NOT EXISTS {schema_role_name_full};{NEW_LINE}" 
        sql_script += f"GRANT ALL PRIVILEGES ON SCHEMA {schema_name} TO DATABASE ROLE {schema_role_name_full} ;{NEW_LINE}" 
    
        sql_script += f"GRANT DATABASE ROLE {schema_role_name_full} TO DATABASE ROLE {DB_ROLE_NAME_FULL};{NEW_LINE}" 
        sql_script += f"GRANT DATABASE ROLE {schema_role_name_write} TO DATABASE ROLE {schema_role_name_full} ;{NEW_LINE}" 
        
    
        sql_script += f"{NEW_LINE}{NEW_LINE}"
        print (sql_script);
    
        if EXECUTE_SCRIPT:
            print (f"--Executing script for schema {schema_name}");
            session.sql("ALTER SESSION SET MULTI_STATEMENT_COUNT = 0").collect()
            session.sql(sql_script).collect()
    

In [ ]:
if script_type == SCRIPT_ALL_SCHEMA and MANAGED_SCHEMA:
    for row in filtered_schemas:
        schema_name = f"{row.name}"
            
        sql_script = f"USE ROLE {ROLE_DATAPRODUCT_OWNER};{NEW_LINE}"
        sql_script += f"USE DATABASE {DATABASE_NAME};{NEW_LINE}"
        sql_script += f"ALTER SCHEMA {DATABASE_NAME}.{row.name} ENABLE MANAGED ACCESS;"
        print (sql_script);
    
        if EXECUTE_SCRIPT:
            session.sql("ALTER SESSION SET MULTI_STATEMENT_COUNT = 0").collect()
            session.sql(sql_script).collect()
            